# Data Cleaning

In [2]:
import sys, os
sys.path.append("../")
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from utils.cleaning import DiscrepancyIdentifier, CleaningFinalDataFrames, cleaning_pipeline

# Data Ingestion

In [3]:
df_register = pd.read_csv("./Data/base_cadastral.csv")
print(f"Shape: {df_register.shape}")
df_info = pd.read_csv("./Data/base_info.csv")
print(f"Shape: {df_info.shape}")
df_payments = pd.read_csv("./Data/base_pagamentos_desenvolvimento.csv")
print(f"Shape: {df_payments.shape}")
df_test = pd.read_csv("./Data/base_pagamentos_teste.csv")
print(f"Shape: {df_test.shape}")

Shape: (1315, 8)
Shape: (24401, 4)
Shape: (77414, 7)
Shape: (12275, 6)


# Base Cadastral

## General Look at the Data

In [4]:
df_register.head()

,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG
0,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65
1,8274986328479596038,2017-01-25,31,NaN,Comércio,YAHOO,MEDIO,77
2,345447888460137901,2000-08-15,75,NaN,Serviços,HOTMAIL,PEQUENO,48
3,1003144834589372198,2017-08-06,49,NaN,Serviços,OUTLOOK,PEQUENO,89
4,324916756972236008,2011-02-14,88,NaN,Serviços,GMAIL,GRANDE,62


In [5]:
df_register.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315 entries, 0 to 1314
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_CLIENTE           1315 non-null   int64 
 1   DATA_CADASTRO        1315 non-null   object
 2   DDD                  1078 non-null   object
 3   FLAG_PF              66 non-null     object
 4   SEGMENTO_INDUSTRIAL  1232 non-null   object
 5   DOMINIO_EMAIL        1285 non-null   object
 6   PORTE                1274 non-null   object
 7   CEP_2_DIG            1315 non-null   object
dtypes: int64(1), object(7)
memory usage: 82.3+ KB


In [6]:
df_register.isna().sum()

ID_CLIENTE                0
DATA_CADASTRO             0
DDD                     237
FLAG_PF                1249
SEGMENTO_INDUSTRIAL      83
DOMINIO_EMAIL            30
PORTE                    41
CEP_2_DIG                 0
dtype: int64

### Observations:
- The dataset contains a lot of missing values
- Only `ID_CLIENTE` is a numerical column
- The column `FLAG_PF` is almost exclusively made of NaN values

### Impact:
- The column `FLAG_PF` can be modified into two categories, removing all NaN values
- `DDD` can be investigated further to see if there's a relation with `CEP_2_DIG` (since these digits indicate the state, like the DDD), so the missing values can be imputed correctly
- `PORTE` can be investigated when joined with the "base info" `NO_FUNCIONARIOS` to see if theres any relation between company size and number of workers
- In all other columns with NaN values, a new category will be made to input the missing values, to avoid any type of miss judgement. I could replace the NaN values with existing values of each column, based on the proportion of them, but since we are dealing with real customer data, I don't think is right or even ethical to just assume something about the client
- It would be interesting to divide the `DDD` and `CEP_2_DIG` columns into the regions that their values indicates

### Analyzing the column `DDD`

In [7]:
# Checking total NaN values
df_register["DDD"].isna().sum()

237

In [8]:
# Checking for non digits characters
non_digit = df_register[df_register["DDD"].str.match(r'^[^\d]', na=False)]
non_digit.head()

,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG
55,2025898153078562640,2007-06-17,(3,NaN,Serviços,GMAIL,PEQUENO,36
70,3218792504195936313,2000-08-15,(2,NaN,Serviços,HOTMAIL,MEDIO,21
124,6759864637454708017,2000-08-15,(6,NaN,Serviços,GMAIL,GRANDE,68
156,1210783884008131648,2000-08-15,(1,NaN,Serviços,YAHOO,MEDIO,13
213,5791518044186095039,2000-08-15,(9,NaN,Serviços,HOTMAIL,PEQUENO,68


In [9]:
non_digit.shape[0]

95

In [10]:
# Checking for discrepancies between DDD and CEP_2_DIG, based on location
analyzer = DiscrepancyIdentifier()
analyzer.identify_discrepancies(df_register)

,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,ESTADO_CEP,ESTADO_DDD
1,8274986328479596038,2017-01-25,31.0,NaN,Comércio,YAHOO,MEDIO,77,TO,MG
6,1276604410881911401,2000-08-15,67.0,NaN,Indústria,YAHOO,PEQUENO,38,MG,MS
8,4530631557358349711,2000-08-15,11.0,NaN,Serviços,HOTMAIL,PEQUENO,55,PE,SP
16,6795467882769787169,2014-06-23,11.0,NaN,Serviços,HOTMAIL,PEQUENO,98,RS,SP
25,1078687383992817574,2000-08-15,43.0,NaN,Indústria,OUTLOOK,GRANDE,67,PA,PR
...,...,...,...,...,...,...,...,...,...,...
1284,1415340753324392369,2004-08-13,11.0,NaN,Comércio,HOTMAIL,MEDIO,23,RJ,SP
1287,2423599281313969582,2016-06-20,11.0,NaN,Serviços,HOTMAIL,MEDIO,61,CE,SP
1289,3965955331400146488,2017-11-29,65.0,X,NaN,YAHOO,MEDIO,78,RO,MT
1290,3156580765759568020,2011-01-31,11.0,NaN,Serviços,AOL,GRANDE,48,BA,SP


### Observations:
- The column has 237 NaN values
- There are 95 registers where there's a non digit character in the `DDD` column
- We have 247 cases of a discrepancy between the registered address and the informed DDD 

### Impact:
- Since there's a lot of cases with location discrepancies, it's not possible to accurately input the missing DDDs based by the `CEP_2_DIG` column
- The same thing applies to the DDDs with non digit characters, where it's impossible to know if the user misspelled a number or if the system automatically types the "(" and the user didn't notice it, so he typed himself
- For these reasons, all DDDs with non digit characters and the missing ones will be replaced with a new category of "Unknown"

### Analyzing the column `PORTE`

In [11]:
# Joining the two dataframses to check for a relation between PORTE and NO_FUNCIONARIOS
merge_registr_info = pd.merge(df_info, df_register, on="ID_CLIENTE", how="left")
merge_registr_info.groupby("PORTE")["NO_FUNCIONARIOS"].agg(["mean", "median", "std", "count", "min", "max"])

,mean,median,std,count,min,max
PORTE,,,,,,
GRANDE,118.770952,119.0,20.549856,8985,0.0,198.0
MEDIO,118.123186,117.0,21.094832,9717,0.0,181.0
PEQUENO,115.158667,117.0,23.814295,5042,0.0,187.0


### Observations:
- The distribution of workers between the different company sizes are almost the same

### Impact:
- An "Unknown" category will have to be created for the `PORTE` column

## Cleaning Pipeline for "Base Cadastral"

### Based on the analysis, is clear that the main problem with this dataset in the amount of missing values. To fix this, the following data cleaning steps will be taken:
- Fill all missing data of `FLAG_PF` with a 0 or 1, with the number 1 representing the "X" values
- Create a category called "Unknown" to fill all missing data of the other columns
- Categorize the columns `DDD` and `CEP_2_DIG` into the 5 regions of the country, for a better EDA and reduced dimentionality for training the model

In [12]:
df_register = pd.read_csv("./Data/base_cadastral.csv")
cleaning_pipeline(df_register, "Data", "clean_base_cadastral")

In [13]:
df_clean = pd.read_csv("./Data/clean_base_cadastral.csv")
df_clean.head()

,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG
0,1661240395903230676,2013-08-22,Nordeste,0,Serviços,YAHOO,PEQUENO,Nordeste
1,8274986328479596038,2017-01-25,Sudeste,0,Comércio,YAHOO,MEDIO,Norte
2,345447888460137901,2000-08-15,Nordeste,0,Serviços,HOTMAIL,PEQUENO,Nordeste
3,1003144834589372198,2017-08-06,Sul,0,Serviços,OUTLOOK,PEQUENO,Sul
4,324916756972236008,2011-02-14,Nordeste,0,Serviços,GMAIL,GRANDE,Nordeste


In [14]:
df_clean.isna().sum()

ID_CLIENTE             0
DATA_CADASTRO          0
DDD                    0
FLAG_PF                0
SEGMENTO_INDUSTRIAL    0
DOMINIO_EMAIL          0
PORTE                  0
CEP_2_DIG              0
dtype: int64

# Base Info

## General Look at the Data

In [15]:
df_info.head()

,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS
0,1661240395903230676,2018-09,16913.0,92.0
1,8274986328479596038,2018-09,106430.0,141.0
2,345447888460137901,2018-09,707439.0,99.0
3,1003144834589372198,2018-09,239659.0,96.0
4,324916756972236008,2018-09,203123.0,103.0


In [16]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24401 entries, 0 to 24400
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_CLIENTE          24401 non-null  int64  
 1   SAFRA_REF           24401 non-null  object 
 2   RENDA_MES_ANTERIOR  24401 non-null  float64
 3   NO_FUNCIONARIOS     24401 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 762.7+ KB


In [17]:
df_info.isna().sum()

ID_CLIENTE            0
SAFRA_REF             0
RENDA_MES_ANTERIOR    0
NO_FUNCIONARIOS       0
dtype: int64

### Observations:
- The dataset is completely clean, with every column with the correct data type and no missing data

# Base Pagamentos Desenvolvimento

## General Look at the Data

In [18]:
df_payments.head()

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA
0,1661240395903230676,2018-08,2018-08-17,2018-09-06,2018-09-06,35516.41,6.99
1,1661240395903230676,2018-08,2018-08-19,2018-09-11,2018-09-10,17758.21,6.99
2,1661240395903230676,2018-08,2018-08-26,2018-09-18,2018-09-17,17431.96,6.99
3,1661240395903230676,2018-08,2018-08-30,2018-10-11,2018-10-05,1341.00,6.99
4,1661240395903230676,2018-08,2018-08-31,2018-09-20,2018-09-20,21309.85,6.99


In [19]:
df_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77414 entries, 0 to 77413
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID_CLIENTE              77414 non-null  int64  
 1   SAFRA_REF               77414 non-null  object 
 2   DATA_EMISSAO_DOCUMENTO  77414 non-null  object 
 3   DATA_PAGAMENTO          77414 non-null  object 
 4   DATA_VENCIMENTO         77414 non-null  object 
 5   VALOR_A_PAGAR           77414 non-null  float64
 6   TAXA                    77414 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 4.1+ MB


In [20]:
df_payments.isna().sum()

ID_CLIENTE                0
SAFRA_REF                 0
DATA_EMISSAO_DOCUMENTO    0
DATA_PAGAMENTO            0
DATA_VENCIMENTO           0
VALOR_A_PAGAR             0
TAXA                      0
dtype: int64

### Observations:
- The dataset is completely clean, with every column with the correct data type and no missing data

# Base Pagamentos Teste

## General Look at the Data

In [21]:
df_test.head()

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA
0,5058298901476893676,2021-07,2021-07-14,2021-08-04,11204.75,4.99
1,274692171162531764,2021-07,2021-07-08,2021-08-23,60718.50,5.99
2,274692171162531764,2021-07,2021-07-11,2021-08-25,60718.50,5.99
3,274692171162531764,2021-07,2021-07-16,2021-08-30,62250.00,5.99
4,465309249432033993,2021-07,2021-07-05,2021-07-30,26593.95,6.99


In [22]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12275 entries, 0 to 12274
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID_CLIENTE              12275 non-null  int64  
 1   SAFRA_REF               12275 non-null  object 
 2   DATA_EMISSAO_DOCUMENTO  12275 non-null  object 
 3   DATA_VENCIMENTO         12275 non-null  object 
 4   VALOR_A_PAGAR           12275 non-null  float64
 5   TAXA                    12275 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 575.5+ KB


In [23]:
df_test.isna().sum()

ID_CLIENTE                0
SAFRA_REF                 0
DATA_EMISSAO_DOCUMENTO    0
DATA_VENCIMENTO           0
VALOR_A_PAGAR             0
TAXA                      0
dtype: int64

### Observations:
- The dataset is completely clean, with every column with the correct data type and no missing data

# Joining the datasets

## Train Dataframe

In [24]:
payments_and_register = pd.merge(df_payments, df_clean, on="ID_CLIENTE", how="left")
payments_final = pd.merge(payments_and_register, df_info, on=["ID_CLIENTE", "SAFRA_REF"], how="left")
payments_final.shape[0]

77414

In [25]:
payments_final.isna().sum()

ID_CLIENTE                   0
SAFRA_REF                    0
DATA_EMISSAO_DOCUMENTO       0
DATA_PAGAMENTO               0
DATA_VENCIMENTO              0
VALOR_A_PAGAR                0
TAXA                         0
DATA_CADASTRO                0
DDD                          0
FLAG_PF                      0
SEGMENTO_INDUSTRIAL          0
DOMINIO_EMAIL                0
PORTE                        0
CEP_2_DIG                    0
RENDA_MES_ANTERIOR        3937
NO_FUNCIONARIOS           3937
dtype: int64

### Observations:
- The left join is used so that no payment data is lost, ensuring that the model can train with all available data
- When the joins are finished, there are missing values on the `RENDA_MES_ANTERIOR` and `NO_FUNCIONARIOS` columns. These values appear because some clients on the payments dataframe are not in the info dataframe for the specific `SAFRA_REF`

### Impact:
- To fix these missing values, it's possible to group the dataset by `ID_CLIENTE` and get the median of `RENDA_MES_ANTERIOR` and `NO_FUNCIONARIOS`, and use these values to replace the NaN ones. The median is used to be less sensible to extreme values and outliers

In [26]:
cleaner = CleaningFinalDataFrames()
columns_to_fill = {'RENDA_MES_ANTERIOR': 'RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS': 'NO_FUNCIONARIOS'}
payments_final = cleaner.fill_missing_values(payments_final, columns_to_fill, payments_final)

In [27]:
payments_final.isna().sum()

ID_CLIENTE                 0
SAFRA_REF                  0
DATA_EMISSAO_DOCUMENTO     0
DATA_PAGAMENTO             0
DATA_VENCIMENTO            0
VALOR_A_PAGAR              0
TAXA                       0
DATA_CADASTRO              0
DDD                        0
FLAG_PF                    0
SEGMENTO_INDUSTRIAL        0
DOMINIO_EMAIL              0
PORTE                      0
CEP_2_DIG                  0
RENDA_MES_ANTERIOR        93
NO_FUNCIONARIOS           93
dtype: int64

### Observations:
- Now, only 93 NaN values remains. These values are from the IDs where all payment data didn't match the `SAFRA_REF` column

### Impact:
- To fix this, it's possible to got through the info dataframe and get the median values from there to replace the missing values here

In [28]:
payments_final = cleaner.fill_missing_values(payments_final, columns_to_fill, df_info)

In [29]:
payments_final.isna().sum()

ID_CLIENTE                0
SAFRA_REF                 0
DATA_EMISSAO_DOCUMENTO    0
DATA_PAGAMENTO            0
DATA_VENCIMENTO           0
VALOR_A_PAGAR             0
TAXA                      0
DATA_CADASTRO             0
DDD                       0
FLAG_PF                   0
SEGMENTO_INDUSTRIAL       0
DOMINIO_EMAIL             0
PORTE                     0
CEP_2_DIG                 0
RENDA_MES_ANTERIOR        0
NO_FUNCIONARIOS           0
dtype: int64

## Test Dataframe

In [30]:
test_and_register = pd.merge(df_test, df_clean, on="ID_CLIENTE", how="left")
test_final = pd.merge(test_and_register, df_info, on=["ID_CLIENTE", "SAFRA_REF"], how="left")
test_final.shape[0]

12275

In [31]:
test_final.isna().sum()

ID_CLIENTE                  0
SAFRA_REF                   0
DATA_EMISSAO_DOCUMENTO      0
DATA_VENCIMENTO             0
VALOR_A_PAGAR               0
TAXA                        0
DATA_CADASTRO              38
DDD                        38
FLAG_PF                    38
SEGMENTO_INDUSTRIAL        38
DOMINIO_EMAIL              38
PORTE                      38
CEP_2_DIG                  38
RENDA_MES_ANTERIOR        414
NO_FUNCIONARIOS           414
dtype: int64

### Observations:
- The same problem as the training dataset appears
- When the joins are finished, there are missing values on the `RENDA_MES_ANTERIOR` and `NO_FUNCIONARIOS` columns. These values appear because some clients on the test dataframe are not in the info dataframe for the specific `SAFRA_REF`
- Additionally, there are missing values on all columns of the register dataframe. This happens because some clients on the test dataframe are not in the register dataframe

### Impact:
- To fix these missing values, it's possible to group the dataset by `ID_CLIENTE` and get the median of `RENDA_MES_ANTERIOR` and `NO_FUNCIONARIOS`, and use these values to replace the NaN ones. The median is used to be less sensible to extreme values and outliers
- To fix the new batch of missing values, the only possible approach is to fill everything with the "Unknown" category, since these clients do not have the information of these columns anywhere else

In [32]:
# Replace with median values
columns_to_fill = {'RENDA_MES_ANTERIOR': 'RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS': 'NO_FUNCIONARIOS'}
test_final = cleaner.fill_missing_values(test_final, columns_to_fill, test_final)
test_final = cleaner.fill_missing_values(test_final, columns_to_fill, df_info)

# Replace with "Unknown" category
columns_register = ['DATA_CADASTRO', 'DDD', 'FLAG_PF', 'SEGMENTO_INDUSTRIAL', 'DOMINIO_EMAIL', 'PORTE', 'CEP_2_DIG']
test_final[columns_register] = test_final[columns_register].fillna("Unknown")

In [33]:
test_final.isna().sum()

ID_CLIENTE                0
SAFRA_REF                 0
DATA_EMISSAO_DOCUMENTO    0
DATA_VENCIMENTO           0
VALOR_A_PAGAR             0
TAXA                      0
DATA_CADASTRO             0
DDD                       0
FLAG_PF                   0
SEGMENTO_INDUSTRIAL       0
DOMINIO_EMAIL             0
PORTE                     0
CEP_2_DIG                 0
RENDA_MES_ANTERIOR        0
NO_FUNCIONARIOS           0
dtype: int64

# Saving the dataframes

In [34]:
payments_final.to_csv("./Data/clean_payments_final", index=False)
test_final.to_csv("./Data/clean_test_final", index=False)